In [ ]:
import numpy as np
import networkx as nx
from numpy.random import rand, randint
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics.cluster import contingency_matrix
from networkx.algorithms import community
import time
import os
import psutil
from collections import defaultdict

# ==============================================================================
# SECTION 1: NEW METRIC CALCULATION FUNCTIONS
# ==============================================================================

def calculate_performance_metrics():
    """Calculates current memory usage of the process."""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    memory_usage = mem_info.rss / (1024 * 1024)  # Convert bytes to MB
    return memory_usage

def calculate_conductance(G, communities):
    """Calculates the average conductance of the partition."""
    if not communities:
        return 0.0
    
    conductance_scores = [nx.conductance(G, comm) for comm in communities if len(comm) > 0 and G.subgraph(comm).number_of_edges() > 0]
    return np.mean(conductance_scores) if conductance_scores else 0.0


def calculate_qds(G, communities):
    """Calculates Modularity Density (Qds) for a partition."""
    m = G.number_of_edges()
    if m == 0:
        return 0.0
    
    qds_val = 0.0
    for community_nodes in communities:
        if not community_nodes:
            continue
        
        # Create a set for faster lookups
        community_set = set(community_nodes)
        
        # Calculate internal and boundary degrees
        internal_degree = 0
        boundary_degree = 0
        
        for node in community_nodes:
            for neighbor in G.neighbors(node):
                if neighbor in community_set:
                    internal_degree += 1
                else:
                    boundary_degree += 1
                    
        mc = internal_degree / 2  # Each internal edge counted twice
        nc = len(community_nodes)
        dc = internal_degree + boundary_degree
        
        if nc > 1:
            internal_density = (2 * mc) / (nc * (nc - 1))
            qds_val += (mc / m) - (dc / (2 * m))**2 * internal_density
            
    return qds_val

def calculate_macro_f1(true_labels_dict, pred_labels_list):
    """
    Calculates the Macro F1 Score between true and predicted communities.
    This is a common approach for comparing clustering results.
    """
    # Convert predicted list of lists to a dictionary for easy lookup
    pred_labels_dict = {node: i for i, comm in enumerate(pred_labels_list) for node in comm}
    
    nodes = list(true_labels_dict.keys())
    true_labels = [true_labels_dict[node] for node in nodes]
    pred_labels = [pred_labels_dict.get(node, -1) for node in nodes]

    cm = contingency_matrix(true_labels, pred_labels)
    
    f1_scores = []
    # Calculate F1 for each true community (row in contingency matrix)
    for i in range(cm.shape[0]):
        true_community_size = np.sum(cm[i, :])
        
        if true_community_size == 0:
            continue

        best_match_j = np.argmax(cm[i, :])
        best_match_size = cm[i, best_match_j]
        predicted_community_size = np.sum(cm[:, best_match_j])
        
        precision = best_match_size / predicted_community_size if predicted_community_size > 0 else 0
        recall = best_match_size / true_community_size
        
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_scores.append(f1)
    
    return np.mean(f1_scores) if f1_scores else 0.0

def calculate_stability(graph, algorithm_class, runs=10, perturbation_rate=0.05, **kwargs):
    """Calculates the stability of the algorithm under edge perturbation."""
    print("\n--- Calculating Stability under Perturbation ---")
    
    # Get the baseline result on the original graph
    algo_instance = algorithm_class(graph, **kwargs)
    base_partition_labels, _ = algo_instance.run() # We only need the labels
    
    nmi_scores = []
    num_edges_to_remove = int(graph.number_of_edges() * perturbation_rate)

    for i in range(runs):
        perturbed_graph = graph.copy()
        
        # Randomly select and remove edges
        if num_edges_to_remove > 0:
            edges_to_remove_indices = np.random.choice(len(perturbed_graph.edges()), num_edges_to_remove, replace=False)
            edges = list(perturbed_graph.edges())
            perturbed_graph.remove_edges_from([edges[i] for i in edges_to_remove_indices])
        
        perturbed_algo = algorithm_class(perturbed_graph, **kwargs)
        perturbed_labels, _ = perturbed_algo.run()
        
        score = nmi(base_partition_labels, perturbed_labels)
        nmi_scores.append(score)
        print(f"Perturbation Run {i+1}/{runs}, NMI vs Original: {score:.4f}")
        
    stability_score = np.mean(nmi_scores)
    print(f"--- Stability Calculation Finished. Average NMI: {stability_score:.4f} ---")
    return stability_score


# ==============================================================================
# SECTION 2: ALGORITHM CLASSES
# ==============================================================================

class Standard_PSO:
    def __init__(self, graph, population_size=200, max_iter=500):
        self.graph = graph
        self.n = len(graph.nodes)
        self.population_size = population_size
        self.max_iter = max_iter
        self.omega = 0.7
        self.c1 = 1.5
        self.c2 = 1.5
        self.population = self.initialize_population()

    def initialize_population(self):
        return np.array([randint(0, self.n, self.n) for _ in range(self.population_size)])

    def calculate_modularity(self, community_assignment):
        return nx.community.modularity(self.graph, self.get_communities(community_assignment))

    def get_communities(self, community_assignment):
        communities = {}
        for node, com in enumerate(community_assignment):
            communities.setdefault(com, []).append(node)
        return list(communities.values())

    def run(self):
        velocities = np.zeros((self.population_size, self.n))
        personal_best = self.population.copy()
        personal_best_scores = np.array([self.calculate_modularity(ind) for ind in self.population])
        global_best = personal_best[np.argmax(personal_best_scores)]
        global_best_score = max(personal_best_scores)

        for iteration in range(self.max_iter):
            r1, r2 = rand(self.population_size, self.n), rand(self.population_size, self.n)
            velocities = (self.omega * velocities + self.c1 * r1 * (personal_best - self.population) 
                          + self.c2 * r2 * (global_best - self.population))
            self.population = np.round(self.population + velocities).astype(int) % self.n
            self.population = np.clip(self.population, 0, self.n - 1)

            fitness = np.array([self.calculate_modularity(ind) for ind in self.population])
            update_mask = fitness > personal_best_scores
            personal_best[update_mask] = self.population[update_mask]
            personal_best_scores[update_mask] = fitness[update_mask]

            if max(fitness) > global_best_score:
                global_best_score = max(fitness)
                global_best = self.population[np.argmax(fitness)]

            if iteration % 10 == 0:
                print(f"Iteration {iteration+1}/{self.max_iter}, Best Modularity: {global_best_score}")
        
        return global_best


class OBPSO_AIW:
    def __init__(self, graph, population_size=150, max_iter=500, ground_truth=None):
        self.graph = graph
        self.n = len(graph.nodes)
        self.population_size = population_size
        self.max_iter = max_iter
        self.omega_max = 0.9
        self.omega_min = 0.2 # Adjusted for better convergence
        self.c1 = 1.2
        self.c2 = 1.8
        self.mutation_rate = 0.25
        self.crossover_rate = 0.9
        self.population = self.initialize_population()
        self.ground_truth = ground_truth # Store ground truth if provided

    # In the OBPSO_AIW class

    def initialize_population(self):
        half_pop = self.population_size // 2
        
        # --- Part 1: Louvain Initialization ---
        initial_partition = community.louvain_communities(self.graph, seed=42)
        community_map = {node: i for i, comm in enumerate(initial_partition) for node in comm}
        nodes = list(self.graph.nodes())
        base_assignment = np.array([community_map[node] for node in nodes])
        louvain_population = [base_assignment.copy() for _ in range(half_pop)]
        
        # --- Part 2: Random Initialization ---
        random_population = [randint(0, self.n, self.n) for _ in range(self.population_size - half_pop)]
        
        # --- Combine them ---
        full_population = louvain_population + random_population
        return np.array(full_population)

    def calculate_modularity(self, community_assignment):
        return nx.community.modularity(self.graph, self.get_communities(community_assignment))

    def get_communities(self, community_assignment):
        communities = defaultdict(list)
        # Ensure we map based on the actual node IDs from the graph
        for i, com_id in enumerate(community_assignment):
            communities[com_id].append(list(self.graph.nodes())[i])
        return list(communities.values())

    def crossover(self, population):
        new_population = []
        for i in range(0, self.population_size, 2):
            parent1 = population[i]
            if i + 1 < self.population_size:
                parent2 = population[i+1]
                if rand() < self.crossover_rate:
                    # Single point crossover
                    crossover_point = randint(1, self.n - 1)
                    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
                    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
                    new_population.extend([child1, child2])
                else:
                    new_population.extend([parent1, parent2])
            else:
                new_population.append(parent1)
        return np.array(new_population)

    # In the OBPSO_AIW class

    def mutate(self, population, iteration):
        dynamic_mutation = self.mutation_rate * (1 - iteration / self.max_iter)
        for individual in population:
            if rand() < dynamic_mutation:
                mutation_point = randint(0, self.n)
                
                # --- NEW AGGRESSIVE MUTATION ---
                # Find the highest existing community ID and add 1 to create a new one.
                # This forces the creation of a new, small community.
                new_community_id = np.max(individual) + 1
                individual[mutation_point] = new_community_id
                
        return population

    def merge_small_communities(self, population):
        for i, individual in enumerate(population):
            unique_comms, counts = np.unique(individual, return_counts=True)
            comm_map = dict(zip(unique_comms, counts))
            
            new_individual = individual.copy()
            for node_idx, comm_id in enumerate(individual):
                if comm_map[comm_id] < 2: # Merge communities with fewer than 3 nodes
                    neighbors = list(self.graph.neighbors(list(self.graph.nodes())[node_idx]))
                    if neighbors:
                        # Find the largest community among neighbors
                        neighbor_comms = [individual[list(self.graph.nodes()).index(n)] for n in neighbors]
                        if neighbor_comms:
                            largest_neighbor_comm = max(set(neighbor_comms), key=neighbor_comms.count)
                            new_individual[node_idx] = largest_neighbor_comm
            population[i] = new_individual
        return population

    def run(self):
        start_time = time.time()
        initial_mem = calculate_performance_metrics()
        
        velocities = np.zeros((self.population_size, self.n))
        personal_best = self.population.copy()
        personal_best_scores = np.array([self.calculate_modularity(ind) for ind in self.population])
        global_best_idx = np.argmax(personal_best_scores)
        global_best = personal_best[global_best_idx]
        global_best_score = personal_best_scores[global_best_idx]

        for iteration in range(self.max_iter):
            omega = self.omega_max - (self.omega_max - self.omega_min) * (iteration / self.max_iter)
            r1, r2 = rand(self.population_size, self.n), rand(self.population_size, self.n)
            
            velocities = (omega * velocities + 
                          self.c1 * r1 * (personal_best - self.population) + 
                          self.c2 * r2 * (global_best - self.population))
            
            self.population = np.round(self.population + velocities).astype(int)
            self.population = np.clip(self.population, 0, self.n - 1)

            self.population = self.merge_small_communities(self.population)
            self.population = self.mutate(self.population, iteration)
            self.population = self.crossover(self.population)

            fitness = np.array([self.calculate_modularity(ind) for ind in self.population])
            
            update_mask = fitness > personal_best_scores
            personal_best[update_mask] = self.population[update_mask]
            personal_best_scores[update_mask] = fitness[update_mask]

            current_best_idx = np.argmax(fitness)
            if fitness[current_best_idx] > global_best_score:
                global_best_score = fitness[current_best_idx]
                global_best = self.population[current_best_idx]

            if iteration % 50 == 0:
                print(f"Iteration {iteration+1}/{self.max_iter}, Best Modularity: {global_best_score:.4f}")

        # --- FINAL METRIC CALCULATION ---
        final_mem = calculate_performance_metrics()
        exec_time = time.time() - start_time
        
        final_communities = self.get_communities(global_best)
        
        results = {
            "Modularity (Q)": global_best_score,
            "Modularity Density (QDS)": calculate_qds(self.graph, final_communities),
            "Conductance": calculate_conductance(self.graph, final_communities),
            "Execution Time (s)": exec_time,
            "Memory Usage (MB)": final_mem - initial_mem,
            "Number of Communities": len(final_communities),
        }

        if self.ground_truth:
            # Align graph nodes for correct label mapping
            node_order = list(self.graph.nodes())
            true_labels = [self.ground_truth[node] for node in node_order]
            
            results["NMI"] = nmi(true_labels, global_best)
            results["ARI"] = ari(true_labels, global_best)
            results["Macro F1 Score"] = calculate_macro_f1(self.ground_truth, final_communities)

        return global_best, results


# ==============================================================================
# SECTION 3: MAIN EXECUTION BLOCK
# ==============================================================================
if __name__ == "__main__":
    
    # --- DATA LOADING FOR EMAIL-EU-CORE ---
    # Define the file paths. (Assumes files are in the same folder as the script)
    edge_file_path = r"D:\pbl\Datasets\email-Eu-core.txt"
    label_file_path = r"D:\pbl\Datasets\email-Eu-core-department-labels.txt"

    print(f"Loading graph from: {edge_file_path}")
    # The graph nodes are integers, so we specify nodetype=int
    G = nx.read_edgelist(edge_file_path, create_using=nx.Graph(), nodetype=int)
    
    # Load the ground truth communities from the labels file
    print(f"Loading ground truth from: {label_file_path}")
    ground_truth_dict = {}
    with open(label_file_path) as f:
        for line in f:
            node, community_id = map(int, line.strip().split()) # Renamed variable
            ground_truth_dict[node] = community_id             # Use the new name
            
    print(f"Graph: email-Eu-core")
    print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
    
    # --- RUN ALGORITHM AND GET RESULTS ---
    print("\n--- Running OBPSO-AIW with Full Evaluation ---")
    
    # Note: This is a larger graph. The process will take longer.
    # You might want to start with fewer iterations (e.g., max_iter=50) for a quick test.
    obpso = OBPSO_AIW(G, population_size=100, max_iter=100, ground_truth=ground_truth_dict)
    
    best_partition_labels, all_results = obpso.run()

    # --- PRINT FINAL RESULTS ---
    print("\n--- ✅ Final Results for email-Eu-core ---")
    for metric, value in all_results.items():
        if isinstance(value, (float, np.floating)):
            print(f"{metric:<25}: {value:.4f}")
        else:
            print(f"{metric:<25}: {value}")
            
    # --- STABILITY CALCULATION (Optional) ---
    #This is very computationally intensive on a larger graph.
     #Consider running with fewer runs and iterations or skipping it initially.
    
    print("\n--- Starting Stability Calculation (this may take a while) ---")
    stability_score = calculate_stability(
         G, 
         OBPSO_AIW, 
         runs=3, # Reduced runs for speed
         population_size=50, 
         max_iter=50, 
        ground_truth=ground_truth_dict
     )
    print(f"{'Stability (Avg. NMI)':<25}: {stability_score:.4f}")

Loading graph from: D:\pbl\Datasets\email-Eu-core.txt
Loading ground truth from: D:\pbl\Datasets\email-Eu-core-department-labels.txt
Graph: email-Eu-core
Nodes: 1005, Edges: 16706

--- Running OBPSO-AIW with Full Evaluation ---
Iteration 1/100, Best Modularity: 0.4323
Iteration 51/100, Best Modularity: 0.4323

--- ✅ Final Results for email-Eu-core ---
Modularity (Q)           : 0.4324
Modularity Density (QDS) : 0.5569
Conductance              : 0.1375
Execution Time (s)       : 456.3414
Memory Usage (MB)        : 5.2227
Number of Communities    : 28
NMI                      : 0.5940
ARI                      : 0.3297
Macro F1 Score           : 0.2456

--- Starting Stability Calculation (this may take a while) ---

--- Calculating Stability under Perturbation ---
Iteration 1/50, Best Modularity: 0.4323
Iteration 1/50, Best Modularity: 0.4334
Perturbation Run 1/3, NMI vs Original: 0.8994
Iteration 1/50, Best Modularity: 0.4333
Perturbation Run 2/3, NMI vs Original: 0.9045
Iteration 1/50, 